In [12]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler

In [13]:
train = pd.read_csv('./Data/training_set_values.csv')
test = pd.read_csv('./Data/training_set_values.csv')

train['train'] = 1
test['train'] = 0
data = pd.concat([train, test])

In [14]:
data.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,train
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,1
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,1
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1


In [15]:
# overview

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118800 entries, 0 to 59399
Data columns (total 41 columns):
id                       118800 non-null int64
amount_tsh               118800 non-null float64
date_recorded            118800 non-null object
funder                   111530 non-null object
gps_height               118800 non-null int64
installer                111490 non-null object
longitude                118800 non-null float64
latitude                 118800 non-null float64
wpt_name                 118800 non-null object
num_private              118800 non-null int64
basin                    118800 non-null object
subvillage               118058 non-null object
region                   118800 non-null object
region_code              118800 non-null int64
district_code            118800 non-null int64
lga                      118800 non-null object
ward                     118800 non-null object
population               118800 non-null int64
public_meeting           1121

In [16]:
# identify missing values in numerical data

int_var = ['population','gps_height','num_private','construction_year']
float_var = ['amount_tsh','longitude']

for var in int_var:
    print('{}:'.format(var))
    display(data[var].min())
    display(len(data[data[var] == 0]))

for var in float_var:
    print('{}:'.format(var))
    display(data[var].min())
    display(len(data[data[var] == 0.0]))

print('latitude:')
display(data['latitude'].min())
display(len(data[(data['latitude'] > -0.001) & (data['latitude'] < 0.001)]))


population:


0

42762

gps_height:


-90

...

0.0

3624

latitude:


-11.64944018

3624

In [17]:
# replace zeros by NaN

for var in int_var:
    data[var].replace(0, np.nan, inplace=True)
    
for var in float_var:
    data[var].replace(0.0, np.nan, inplace=True)

data['latitude'].where((data['latitude'] < -0.001) | (data['latitude'] > 0.001), other= np.nan, inplace=True,axis=0)

In [18]:
data.isnull().sum()

id                            0
amount_tsh                83278
date_recorded                 0
funder                     7270
gps_height                40876
installer                  7310
longitude                  3624
latitude                   3624
wpt_name                      0
num_private              117286
basin                         0
subvillage                  742
region                        0
region_code                   0
district_code                 0
lga                           0
ward                          0
population                42762
public_meeting             6668
recorded_by                   0
scheme_management          7754
scheme_name               56332
permit                     6112
construction_year         41418
extraction_type               0
extraction_type_group         0
extraction_type_class         0
management                    0
management_group              0
payment                       0
payment_type                  0
water_qu

In [19]:
# Duplicate longitude and latitude column for imputation based on normal distribution and random choice
data['longitude_imp_normal'] = data['longitude']
data['latitude_imp_normal'] = data['latitude']
data['longitude_imp_rand_choice'] = data['longitude']
data['latitude_imp_rand_choice'] = data['latitude']

In [20]:
# Add columns for mean and standard deviation of longitude and latitude based on region and ward
gps_features = ['longitude', 'latitude']
divisions = ['region', 'ward']
measures = ['mean', 'std']
for gps_feature in gps_features:
    for division in divisions:
        for measure in measures:
            new_feature_name = '_'.join([gps_feature, measure, division])
            data[new_feature_name] = data.groupby(division)[gps_feature].transform(measure)

In [21]:
# 1st step: Impute missing values with random numbers generated by normal distribution based on mean, std by ward
data['longitude_imp_normal'] = data.apply(lambda row: np.random.normal(loc=row['longitude_mean_ward'], scale=row['longitude_std_ward']) if math.isnan(row['longitude_imp_normal']) else row['longitude_imp_normal'], axis=1)
data['latitude_imp_normal'] = data.apply(lambda row: np.random.normal(loc=row['latitude_mean_ward'], scale=row['latitude_std_ward']) if math.isnan(row['latitude_imp_normal']) else row['latitude_imp_normal'], axis=1)

display('Missing values after imputation by ward: {}'.format(data['longitude_imp_normal'].isnull().sum()))

# 12nd step: Impute missing values with random numbers generated by normal distribution based on mean, std by region
data['longitude_imp_normal'] = data.apply(lambda row: np.random.normal(loc=row['longitude_mean_region'], scale=row['longitude_std_region']) if math.isnan(row['longitude_imp_normal']) else row['longitude_imp_normal'], axis=1)
data['latitude_imp_normal'] = data.apply(lambda row: np.random.normal(loc=row['latitude_mean_region'], scale=row['latitude_std_region']) if math.isnan(row['latitude_imp_normal']) else row['latitude_imp_normal'], axis=1)


'Missing values after imputation by ward: 2916'

In [22]:
# Add columns with list of values in corresponding group of region and ward, respectively
for gps_feature in gps_features:
    for division in divisions:
        feature_name = '_'.join([gps_feature, 'list', division])
        lists = data.groupby(division)[gps_feature].apply(list)
        data[feature_name] = data.apply(lambda row: lists[row[division]], axis=1)
        data[feature_name] = data[feature_name].apply(lambda lst: [x for x in lst if not math.isnan(x)])
        data[feature_name] = data[feature_name].apply(lambda x: 1 if not x else x)
        display(gps_feature, division)

'longitude'

'region'

'longitude'

'ward'

'latitude'

'region'

'latitude'

'ward'

In [24]:
data['longitude_imp_rand_choice'] = data['longitude']
data['latitude_imp_rand_choice'] = data['latitude']

In [26]:
# 1st step: Impute missing values with random numbers generated by random choice on list of not-nan values per ward
data['longitude_imp_rand_choice'] = data.apply(lambda row: np.random.choice(a=row['longitude_list_ward']) if math.isnan(row['longitude_imp_rand_choice']) else row['longitude_imp_rand_choice'], axis=1)
data['latitude_imp_rand_choice'] = data.apply(lambda row: np.random.choice(a=row['latitude_list_ward']) if math.isnan(row['latitude_imp_rand_choice']) else row['latitude_imp_rand_choice'], axis=1)

display('Missing values after imputation by ward: {}'.format(len(data.loc[data['longitude_imp_rand_choice'] == 0])))

# 2nd step: Impute missing values with random numbers generated by random choice on list of not-nan values per region
data['longitude_imp_rand_choice'] = data.apply(lambda row: np.random.choice(a=row['longitude_list_region']) if row['longitude_imp_rand_choice'] == 0 else row['longitude_imp_rand_choice'], axis=1)
data['latitude_imp_rand_choice'] = data.apply(lambda row: np.random.choice(a=row['latitude_list_region']) if row['latitude_imp_rand_choice'] == 0 else row['latitude_imp_rand_choice'], axis=1)


'Missing values after imputation by ward: 2916'

In [27]:
# Drop columns used for generation of random numbers
drop_columns = list()
measures.append('list')
for gps_feature in gps_features:
    for division in divisions:
        for measure in measures:
            drop_columns.append('_'.join([gps_feature, measure, division]))
data.drop(columns=drop_columns, inplace=True)

In [28]:
# fill numerical null-values by mean/median

int_var.append('latitude')

# 1st step: group by region, ward (-> null-values remain if there does not exist a single non-null value in a tuple)
for var in int_var:
    data[var].fillna(data.groupby(['region', 'ward'])[var].transform("median"), inplace=True)

for var in float_var:
    data[var].fillna(data.groupby(['region', 'ward'])[var].transform("mean"), inplace=True)
    
# 2nd step: rougher filter
for var in int_var:
    data[var].fillna(data.groupby(['region'])[var].transform("median"), inplace=True)

for var in float_var:
    data[var].fillna(data.groupby(['region'])[var].transform("mean"), inplace=True)

# 3rd step: rougher filter
for var in int_var:
    data[var].fillna(data[var].median(), inplace=True)

for var in float_var:
    data[var].fillna(data[var].mean(), inplace=True)

In [29]:
data.isnull().sum()

id                               0
amount_tsh                       0
date_recorded                    0
funder                        7270
gps_height                       0
installer                     7310
longitude                        0
latitude                         0
wpt_name                         0
num_private                      0
basin                            0
subvillage                     742
region                           0
region_code                      0
district_code                    0
lga                              0
ward                             0
population                       0
public_meeting                6668
recorded_by                      0
scheme_management             7754
scheme_name                  56332
permit                        6112
construction_year                0
extraction_type                  0
extraction_type_group            0
extraction_type_class            0
management                       0
management_group    

In [30]:
# create new feature that gives information about operational time
    
data['date_recorded'] = pd.to_datetime(data['date_recorded'])
data['operation_years'] = data.date_recorded.dt.year - data.construction_year
data['operation_years'] = data['operation_years'].astype(int)

In [ ]:
# scale numeric features

#num_features=['latitude','longitude','operation_years','amount_tsh', 'gps_height', 'population']
#scaler = MinMaxScaler()

#for s in split:
    #s[num_features] = scaler.fit_transform(s[num_features])

In [31]:
# drop redundant features and features that do not seem to have an impact

data.drop(['extraction_type_group','extraction_type_class','payment','quality_group','source_class','source_type','waterpoint_type_group','management_group','quantity_group','date_recorded','wpt_name','num_private','recorded_by'],axis=1,inplace=True)

In [32]:
data['funder'].value_counts()
data['installer'].value_counts()
data['scheme_name'].value_counts()

K                                        1364
None                                     1288
Borehole                                 1092
Chalinze wate                             810
M                                         800
DANIDA                                    758
Government                                640
Ngana water supplied scheme               540
wanging'ombe water supply s               522
wanging'ombe supply scheme                468
Bagamoyo wate                             458
I                                         458
Uroki-Bomang'ombe water sup               418
N                                         408
Kirua kahe gravity water supply trust     386
Machumba estate pipe line                 370
Makwale water supplied sche               332
Kijiji                                    322
S                                         308
mtwango water supply scheme               304
Losaa-Kia water supply                    304
Handeni Trunk Main(H              

In [33]:
# reduce dimension of categorical variables

# funder
data = data.assign(count = data.groupby('funder')['funder'].transform('count'))\
.sort_values(by = ['count','funder'], ascending = [False,True])

data.loc[data['count'] < 1050, 'funder'] = 'Others'
data['funder'].replace(np.nan, 'Others', inplace=True)
del data['count']

# installer
data = data.assign(count = data.groupby('installer')['installer'].transform('count'))\
.sort_values(by = ['count','installer'], ascending = [False,True])

data.loc[data['count'] < 765, 'installer'] = 'Others'
data.loc[data['installer'] == '0', 'installer'] = 'Others'
data['installer'].replace(np.nan, 'Others', inplace=True)
del data['count']

# scheme_name
data = data.assign(count = data.groupby('scheme_name')['scheme_name'].transform('count'))\
.sort_values(by = ['count','scheme_name'], ascending = [False,True])

data.loc[data['count'] < 296, 'scheme_name'] = 'Others'
data.loc[data['scheme_name'] == '0', 'scheme_name'] = 'Others'
data['scheme_name'].replace(np.nan, 'Others', inplace=True)
del data['count']

In [34]:
# factorize features for evaluations

#data['funder'] = pd.factorize(data['funder'])[0]
#data['installer'] = pd.factorize(data['installer'])[0]
#data['basin'] = pd.factorize(data['basin'])[0]
#data['subvillage'] = pd.factorize(data['subvillage'])[0]
#data['region'] = pd.factorize(data['region'])[0]
#data['lga'] = pd.factorize(data['lga'])[0]
#data['ward'] = pd.factorize(data['ward'])[0]
#data['scheme_management'] = pd.factorize(data['scheme_management'])[0]
#data['scheme_name'] = pd.factorize(data['scheme_name'])[0]
#data['extraction_type'] = pd.factorize(data['extraction_type'])[0]
#data['management'] = pd.factorize(data['management'])[0]
#data['payment_type'] = pd.factorize(data['payment_type'])[0]
#data['water_quality'] = pd.factorize(data['water_quality'])[0]
#data['quantity'] = pd.factorize(data['quantity'])[0]
#data['waterpoint_type'] = pd.factorize(data['waterpoint_type'])[0]
#data['permit'] = pd.factorize(data['permit'])[0]
#data['source'] = pd.factorize(data['source'])[0]

In [35]:
data.head()

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,subvillage,region,...,water_quality,quantity,source,waterpoint_type,train,longitude_imp_normal,latitude_imp_normal,longitude_imp_rand_choice,latitude_imp_rand_choice,operation_years
325,21669,812.5,Others,1715.0,DWE,31.366025,-7.972010,Lake Tanganyika,Katapulo,Rukwa,...,soft,enough,spring,communal standpipe,1,31.366025,-7.972010,31.366025,-7.972010,37
11364,60108,1250.0,Others,1906.0,DWE,31.431333,-7.850336,Lake Tanganyika,No 1,Rukwa,...,soft,enough,river,communal standpipe,1,31.431333,-7.850336,31.431333,-7.850336,35
14462,44820,812.5,Others,1741.0,DWE,31.368317,-7.971081,Lake Tanganyika,Muungano,Rukwa,...,soft,enough,spring,communal standpipe,1,31.368317,-7.971081,31.368317,-7.971081,37
18159,53307,812.5,Others,1730.0,DWE,31.368730,-7.972788,Lake Tanganyika,Muungano,Rukwa,...,soft,enough,spring,communal standpipe,1,31.368730,-7.972788,31.368730,-7.972788,37
18990,5701,812.5,Others,1731.0,DWE,31.365461,-7.970985,Lake Tanganyika,Ilyema,Rukwa,...,soft,enough,spring,communal standpipe,1,31.365461,-7.970985,31.365461,-7.970985,37


In [38]:
train_df = data[data["train"] == 1]
test_df = data[data["train"] == 0]

train_df.drop(["train"], axis=1, inplace=True)
test_df.drop(["train"], axis=1, inplace=True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [39]:
pd.DataFrame(train_df).to_csv("./Data/train_cleaned_distr-imp.csv", index=False)
pd.DataFrame(test_df).to_csv("./Data/test_cleaned_distr_imp.csv", index=False)